![CH8-ADS.png](.\Media\CH8-ADS.png)
# <span style="color:#cc5500;">Deploy Azure SQL VM from the Azure Portal </span>

There are several ways to deploy a net-new SQL Server to Azure VM's.  In the following notebooks we will cover most scenarios on how you would deploy SQL Server to the cloud:

- Deploy Azure SQL VM's using the Azure Portal - Covered in this notebook
- Deploy Azure SQL VM's using Power Shell - Covered in the next notebook
- Deploy Azure SQL VM's using an ARM Template - Covered in a later notebook

The 3 bullets above are for a net-new SQL Server on Azure VM.  The simplist form to just get you used to using Azure and seeing how this work.  In a later notebook we will do more of a make-it-real-scenario and show you how to migrate an actual on-premises production SQL Server to Azure.

  

At this time, I do not plan to cover how to deploy an Azure VM from a blank Operating System, configure all proprietary software and then how to upload SQL Server Media and to a clean install of SQL on the VM's.  That process is very lengthy and for the purpose of this book, while still be covering a tremendous amount of SQL Server subjects, but at the same time, keeping it simple when possible.

  

It is assumed that you already have either a Paid subscription or a Trial Subscription.  If you do not, you can go to [Create Your Azure Free Account Today | Microsoft Azure](https://azure.microsoft.com/en-us/free/)

  

Login to the Commercial Azure Portal here:  https://portal.azure.com/  

- Everyone reading this notebook that has an Azure Subscription, has access to Azure Commercial

  

As an alternative, I will be using Microsoft Azure Government by logging in to:

- For those members that have a subscription in an Azure Government GCC Tenant, you can connect to [Home - Microsoft Azure Government](https://portal.azure.us/#home)
- Many of my samples will show Azure Government GCC because there are generally a few more steps for you to take to get our tools like SSMS, Visual Studio Code, Azure Data Studio to connect to Government Clouds, and I want to demonstrate that for you when possible.

## <span style="color:#cc5500;">Choose a Market Place Image</span>

Use the Azure Marketplace to choose one of several pre-configured images from the virtual machine gallery.  Again, to keep this as simple as possible, we are going to use a image already in the Gallery. 

The Developer edition is used in this notebook because it is a full-featured, free edition of SQL Server for development testing.  You pay only for the cost of running the VM.  However, you are free to choose any of the images to use in this walkthrough. For a description of available images, see this link: [Overview of SQL Server on Azure Windows Virtual Machines - SQL Server on Azure VMs | Microsoft Docs](https://docs.microsoft.com/en-us/azure/azure-sql/virtual-machines/windows/sql-server-on-azure-vm-iaas-what-is-overview?view=azuresql#payasyougo)

Licensing costs for SQL Server are incorporated into the per-second pricing of the VM you create and varies by edition and cores. However, SQL Server Developer edition is free for development and testing, not production. Also, SQL Express is free for lightweight workloads (less than 1 GB of memory, less than 10 GB of storage). You can also bring-your-own-license (BYOL) and pay only for the VM. Those image names are prefixed with {BYOL}. For more information on pricing see this link: [Price guidance & managing costs - SQL Server on Azure VMs | Microsoft Docs](https://docs.microsoft.com/en-us/azure/azure-sql/virtual-machines/windows/pricing-guidance?view=azuresql)

To choose an image, follow these steps:

1. From the home page of the Portal click the Marketplace Icon, or in the search bar enter "Marketplace" and then click the icon when it appears.  The UI of the Portal changes often, so your view may be slightly different.  Click Create Virutal Machine

    
    ![CH8-Gallery.jpg](.\Media\CH8-Gallery.jpg)
    
2. Select Azure SQL in the left-hand menu of the Azure portal. If Azure SQL is not in the list, select All services, then type Azure SQL in the search box. You can select the star next to Azure SQL to save it as a favorite to pin it to the left-hand navigation.

    
    ![CH8-ChooseImage.jpg](.\Media\CH8-ChooseImage.jpg)

    
3. Select + Create to open the Select SQL deployment option page. Select the Image drop-down and then select an image.  I choose Free SQL Server License: SQL Server 2019 on wondws Server 2022.  You can also choose Show details for additional information about the image.

## <span style="color:#cc5500;">Basic Settings</span>

The Basics tab allows you to select the subscription, resource group, and instance details.

Using a new resource group is helpful if you are just testing or learning about SQL Server deployments in Azure.  After you finish with your test, delete the resource group to automatically delete the VM and all resources associated with that resource group. For more information about resource groups, see Azure Resource Manager Overview.

On the Basics tab, provide the following information:

- Under Project Details, make sure the correct subscription is selected.
- In the Resource group section, either select an existing resource group from the list or choose Create new to create a new resource group. A resource group is a collection of related resources in Azure (virtual machines, storage accounts, virtual networks, etc.).

Under Instance details:

- Enter a unique Virtual machine name.
- Choose a location for your Region.
- For the purpose of this guide, leave Availability options set to No infrastructure redundancy required. To find out more information about availability options, see Availability.
- In the Image list, select Free SQL Server License: SQL Server 2019 Developer on Windows Server 2019 if it's not already selected.
- Choose Standard for Security type.
- Select See all sizes for the Size of the virtual machine and choose a size.  I choose a small one for this demo.  If this is for testing purposes, be sure to clean up your resources once you're done with them to prevent any unexpected charges
- Under Inbound port rules, choose Allow selected ports and then select RDP (3389) from the drop-down so that you can RDP to the server after it has been created

![CH8-Basics.jpg](.\Media\CH8-Basics.jpg)

## <span style="color:#cc5500;">Discs</span>

On the Disks tab, configure your disk options.

- Under OS disk type, select the type of disk you want for your OS from the drop-down. Premium is recommended for production systems but is not available for a Basic VM. To use a Premium SSD, change the virtual machine size. For the purpose of this demo, I choose Standard SSD
- Under Advanced, select Yes under use Managed Disks.

Microsoft recommends Managed Disks for SQL Server. Managed Disks handles storage behind the scenes. In addition, when virtual machines with Managed Disks are in the same availability set, Azure distributes the storage resources to provide appropriate redundancy. For more information, see Azure Managed Disks Overview. For specifics about managed disks in an availability set, see Use managed disks for VMs in availability set.

## <span style="color:#cc5500;">Networking</span>

On the Networking tab, configure your networking options.

- Create a new virtual network or use an existing virtual network for your SQL Server VM. Designate a Subnet as well.
- Under NIC network security group, select either a basic security group or the advanced security group. Choosing the basic option allows you to select inbound ports for the SQL Server VM which are the same values configured on the Basic tab. Selecting the advanced option allows you to choose an existing network security group, or create a new one.
- You can make other changes to network settings, or keep the default values.  For this demo, I just use the default values

## <span style="color:#cc5500;">Management</span>

On the Management tab, configure monitoring and auto-shutdown.

- Azure enables Boot diagnostics by default with the same storage account designated for the VM. On this tab, you can change these settings and enable OS guest diagnostics.
- You can also enable System assigned managed identity and auto-shutdown on this tab.
- For this demo, I select my timezone and accept all remaining default options
- I Skip the Advanced tab and go directly to the SQL Server Settings Tab next

## <span style="color:#cc5500;">SQL Server settings</span>

On the SQL Server settings tab, configure specific settings and optimizations for SQL Server. You can configure the following settings for SQL Server:

### <span style="color:#cc5500;">Connectivity</span>

Under SQL connectivity, specify the type of access you want to the SQL Server instance on this VM. For the purposes this demo, select Public (internet) to allow connections to SQL Server from machines or services on the internet. With this option selected, Azure automatically configures the firewall and the network security group to allow traffic on the port selected.  To connect to SQL Server via the internet, you also must enable SQL Server Authentication, which is described in the next section.

If you would prefer to not enable connections to the Database Engine via the internet, choose one of the following options:

- Local (inside VM only) to allow connections to SQL Server only from within the VM.
- Private (within Virtual Network) to allow connections to SQL Server from machines or services in the same virtual network.

In general, improve security by choosing the most restrictive connectivity that your scenario allows. But all the options are securable through network security group (NSG) rules and SQL/Windows Authentication. You can edit the NSG after the VM is created

### <span style="color:#cc5500;">Authentication</span>

If you require SQL Server Authentication, select Enable under SQL Authentication on the SQL Server settings tab.   If you enable SQL Server Authentication, specify a Login name and Password. This login name is configured as a SQL Server Authentication login and a member of the sysadmin fixed server role. For more information about Authentication Modes, see Choose an Authentication Mode.  [Choose an Authentication Mode - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/relational-databases/security/choose-an-authentication-mode?view=sql-server-ver15)

If you prefer not to enable SQL Server Authentication, you can use the local Administrator account on the VM to connect to the SQL Server instance.

### <span style="color:#cc5500;">Azure Key Vault integration</span>

To store security secrets in Azure for encryption, select SQL Server settings, and scroll down to Azure key vault integration. Select Enable and fill in the requested information.

Learn more about AKV here:  [Integrate Key Vault with SQL Server on Windows VMs in Azure (Resource Manager) - SQL Server on Azure VMs | Microsoft Docs](https://docs.microsoft.com/en-us/azure/azure-sql/virtual-machines/windows/azure-key-vault-integration-configure?view=azuresql)

### <span style="color:#cc5500;">Storage configuration</span>

- On the SQL Server settings tab, under Storage configuration, select Change configuration to open the Configure storage page and specify storage requirements. You can choose to leave the values at default, or you can manually change the storage topology to suit your IOPS needs
- Under Data storage, choose the location for your data drive, the disk type, and the number of disks. You can also select the checkbox to store your system databases on your data drive instead of the local C:\\ drive.
- Under Log storage, you can choose to use the same drive as the data drive for your transaction log files, or you can choose to use a separate drive from the drop-down. You can also choose the name of the drive, the disk type, and the number of disks.
- Configure your tempdb database settings under Tempdb storage, such as the location of the database files, as well as the number of files, initial size, and autogrowth size in MB. Currently, the max number of tempdb files. Currently, during deployment, the max number of tempdb files is 8, but more files can be added after the SQL Server VM is deployed.

### <span style="color:#cc5500;">SQL instance settings</span>

Select Change SQL instance settings to modify SQL Server configuration options, such as the server collation, max degree of parallelism (MAXDOP), SQL Server min and max memory limits, and whether you want to optimize for ad-hoc workloads.

### <span style="color:#cc5500;">SQL Server Licence</span>

If you're a Software Assurance customer, you can use the Azure Hybrid Benefit to bring your own SQL Server license and save on resources. Select Yes to enable the Azure Hybrid Benefit, and then confirm that you have Software Assurance by selecting the checkbox.

If you chose a free license image, such as the developer edition, the SQL Server license option is grayed out

### <span style="color:#cc5500;">Automated patching</span>

Automated patching is enabled by default. Automated patching allows Azure to automatically apply SQL Server and operating system security updates.  Specify a day of the week, time, and duration for a maintenance window. Azure performs patching in this maintenance window. The maintenance window schedule uses the VM locale. If you do not want Azure to automatically patch SQL Server and the operating system, select Disable.  Learn more here: [Automated Patching for SQL Server VMs (Resource Manager) - SQL Server on Azure VMs | Microsoft Docs](https://docs.microsoft.com/en-us/azure/azure-sql/virtual-machines/windows/automated-patching?view=azuresql)

### <span style="color:#cc5500;">Automated backup</span>

Enable automatic database backups for all databases under Automated backup. Automated backup is disabled by default.

When you enable SQL automated backup, you can configure the following settings:

- Retention period for backups (up to 90 days)
- Storage account, and storage container, to use for backups
- Encryption option and password for backups
- Backup system databases
- Configure backup schedule

To encrypt the backup, select Enable. Then specify the Password. Azure creates a certificate to encrypt the backups and uses the specified password to protect that certificate.

Choose Select Storage Container to specify the container where you want to store your backups.

By default the schedule is set automatically, but you can create your own schedule by selecting Manual, which allows you to configure the backup frequency, backup time window, and the log backup frequency in minutes.  Learn more here: [Automated Backup for SQL Server 2014 Azure virtual machines - SQL Server on Azure VMs | Microsoft Docs](https://docs.microsoft.com/en-us/azure/azure-sql/virtual-machines/windows/automated-backup-sql-2014?view=azuresql)

### <span style="color:#cc5500;">Machine Learning Services</span>

You have the option to enable Machine Learning Services. This option lets you use machine learning with Python and R in SQL Server 2017. Select Enable on the SQL Server Settings window.  Learn more here: [SQL machine learning documentation - Tutorials and install guides | Microsoft Docs](https://docs.microsoft.com/en-us/sql/machine-learning/?view=sql-server-ver15)

## <span style="color:#cc5500;">Review and Create</span>

![CH8-Review.jpg](.\Media\CH8-Review.jpg)

After doing a review of your settings, select Create

Depending on the choices you made, has an impact on the amount of time it takes for your SQL Server on Azure VM to finish deployment.  In my case, with the options I chose, took about xx minutes.

## <span style="color:#cc5500;">Review the Deployment Results</span>

Review the details of the deployment and ensure all dependent object have been created.

![CH8-Results.jpg](.\Media\CH8-Results.jpg)

## <span style="color:#cc5500;">Test Connectivity to your SQL Server on Azure VM using RDP</span>

Open the VM with Remote Desktop

Use the following steps to connect to the SQL Server virtual machine with Remote Desktop Protocol (RDP):

1. After the Azure virtual machine is created and running, click the Virtual Machines icon in the Azure portal to view your VMs.
2. Click the ellipsis, ..., for your new VM.
3. Click Connect.
4. Open the RDP file that your browser downloads for the VM.
5. The Remote Desktop Connection notifies you that the publisher of this remote connection cannot be identified. Click Connect to continue.
6. In the Windows Security dialog, click Use a different account. You might have to click More choices to see this. Specify the user name and password that you configured when you created the VM. You must add a backslash before the user name.
7. Click OK to connect.

After you connect to the SQL Server virtual machine, you can launch SQL Server Management Studio and connect with Windows Authentication using your local administrator credentials. If you enabled SQL Server Authentication, you can also connect with SQL Authentication using the SQL login and password you configured during provisioning.

Access to the machine enables you to directly change machine and SQL Server settings based on your requirements. For example, you could configure the firewall settings or change SQL Server configuration settings.

## <span style="color:#cc5500;">Test Connectivity to your SQL Server on Azure VM using a Remote Connection</span>

In this walkthrough, you most likely selected Public access, (as I did) for the virtual machine and SQL Server Authentication. These settings automatically configured the virtual machine to allow SQL Server connections from any client over the internet (assuming they have the correct SQL login).  If you did not select Public during provisioning, then you can change your SQL connectivity settings through the portal after provisioning. For more information: [Connect to a SQL Server virtual machine (Resource Manager) - SQL Server on Azure VMs | Microsoft Docs](https://docs.microsoft.com/en-us/azure/azure-sql/virtual-machines/windows/ways-to-connect-to-sql?view=azuresql#change)

Configure a DNS Label for the public IP address of your Azure VM (Optional)

To connect to the SQL Server Database Engine from the Internet, consider creating a DNS Label for your public IP address. You can connect by IP address, but the DNS Label creates an A Record that is easier to identify and abstracts the underlying public IP address.  Note:  DNS Labels are not required if you plan to only connect to the SQL Server instance within the same Virtual Network or only locally.

To create a DNS Label:

1. Select Virtual machines in the portal Select your SQL Server VM to bring up its properties.
2. In the virtual machine overview, select your Public IP address.
3. In the properties for your Public IP address, expand Configuration.
4. Enter a DNS Label name. This name is an A Record that can be used to connect to your SQL Server VM by name instead of by IP Address directly.
5. Click the Save button

  

![CH8-DNSlabel.jpg](.\Media\CH8-DNSlabel.jpg)

1. On a computer connected to the internet, open SQL Server Management Studio (SSMS). If you do not have SQL Server Management Studio, you can download it here [Download SQL Server Management Studio (SSMS) - SQL Server Management Studio (SSMS) | Microsoft Docs](https://docs.microsoft.com/en-us/sql/ssms/download-sql-server-management-studio-ssms?view=sql-server-ver15)
2. In the Connect to Server or Connect to Database Engine dialog box, edit the Server name value. Enter the IP address or full DNS name of the virtual machine (determined in the previous task). You can also add a comma and provide SQL Server's TCP port. For example in my case it is:    prodsqlsrv1.usgovvirginia.cloudapp.usgovcloudapi.net,1433.      If you didn't create a DNS lable in the previous step, they you would use the IP Address instead of the DNS Label.
3. In the Authentication box, select SQL Server Authentication.
4. In the Login box, type the name of a valid SQL login.
5. In the Password box, type the password of the login.
6. Click Connect

![CH8-SSMS.jpg](.\Media\CH8-SSMS.jpg)